# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/root/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 7/7 [01:26<00:00, 12.37s/it]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

2024-01-28 20:20:46.723769: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 20:20:46.723831: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 20:20:46.723873: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 20:20:47.056719: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 20:20:49.142903: W tensorflow/c

In [7]:
print(response)

乾卦是八卦之一，也是八宫图中的第一个卦。乾卦是由两个阴爻夹一个阳爻构成，象征着天、干燥、刚强、积极、进取等含义。

乾卦的卦辞是“元、亨、利、贞”，其中“元”表示天地之始，代表一切事物的根源和初始；“亨”表示通，代表顺利、畅通无阻；“利”表示有益、有利，代表收获和收益；“贞”表示正，代表正确、正宗。

在八宫图中，乾卦位于北方，与事业、努力、刚强、积极、进取等有关，同时也与领导、权力、决策等方面有关。在命理学中，乾卦被认为具有积极向上、勇往直前、坚定不移的品质，对于追求成功和进步的人来说，乾卦是一个积极的象征。


In [8]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是八卦之一，由两个阴爻夹一个阳爻构成，象征着地、水、军队等含义。

地水师卦的卦辞是“元、亨、利、贞”，其中“元”表示天地之始，代表一切事物的根源和初始；“亨”表示通，代表顺利、畅通无阻；“利”表示有益、有利，代表收获和收益；“贞”表示正，代表正确、正宗。

在八宫图中，地水师卦位于北方，与事业、努力、刚强、积极、进取等有关，同时也与军队、领导、权力等方面有关。在命理学中，地水师卦被认为具有团结、合作、组织、协调的品质，对于追求成功和进步的人来说，地水师卦是一个积极的象征。此外，地水师卦还与事业发展、组织管理、团队合作等方面有关，具有指导意义。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [10]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [11]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表着天，它由六条阳爻（代表刚强）组成，分为三个阳爻的卦（上）和三个阳爻的卦（中），以及三个阳爻的卦（下）。这种卦象象征着天象的运行，具有刚强的力量。在卜筮时，阳爻表示吉利，阴爻表示凶险。

乾卦的卦辞为：“元、亨、利、贞”，意味着大吉大利，刚强有力。在卜筮时，阳爻表示吉利，阴爻表示凶险。在运势最佳时，应该积极争取，Expand your horizons.在运势 worst时，应该冷静分析，从阳爻变为阴爻，从刚强变为柔顺，从吉凶变为凶险。在运势平顺时，则应 shaped "sharp" (尖) "branch" (枝)，并期望 "moderate" (中) "柔顺" "happy" (愉快) "calm" (平和) "favoring" (有利) "furtherance" (进步) "advance" (前进) "reform" (改革) "uplifting" (提高) "power" (力量) "victory" (胜利)".

在事业、经商、求名、婚恋、决策等方面，都具有刚强的力量和强烈的吉利意义，需要 shaped "sharp" (尖) "branch" (枝)，并期望 "moderate" (中) "柔顺" "happy" (愉快) "calm" (平和) "favoring" (有利) "furtherance" (进步) "advance" (前进) "reform" (改革) "uplifting" (提高) "power" (力量) "victory" (胜利) "promise" (承诺) "danger" (危险) "difficulty" (困难) "dissatisfaction" (不满意) "humility" (谦虚) "failure" (失败) "regret" (后悔) " bitterness" (苦涩) "anger" (愤怒) "worry" (担忧) "fear" (害怕) "sorrow" (悲伤) "despair" (绝望) "希望" (希望) "gratitude" (感激) "mercy" (慈悲) "benevolence" (仁慈) "vitality" (活力) "virtue" (品德) "vitality" (活力) "virtue" 

In [12]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由坎卦（水）上承坤卦（地），代表地上水流之象。在卦象中，上卦为坤卦，下卦为坎卦。坤卦代表地，象征柔顺、顺应，坎卦代表水，象征灵活、变化。地水师卦的卦象代表了一种适应环境变化、顺应地性的态度，象征着事物运动的规律。

地水师卦的卦象变化为：初盘：坤卦（地），坎卦（水），代表地面上有水流。

再见：坤卦（地），坎卦（水），代表地上的水流仍在流动。

复见：坤卦（地），坎卦（水），代表地上的水流仍在流动。

宋代：坤卦（地），坎卦（水），代表地上的水流仍在流动。

当代：坤卦（地），坎卦（水），代表地上的水流仍在流动。

变卦：初盘：坤卦（地），坎卦（水），代表地面上有水流。

再见：坤卦（地），坎卦（水），代表地上的水流已流到地面。

复见：坤卦（地），坎卦（水），代表地上的水流仍在流动。

宋代：坤卦（地），坎卦（水），代表地上的水流仍在流动。

当代：坤卦（地），坎卦（水），代表地上的水流仍在流动。

谦易：初盘：坤卦（地），坎卦（水），代表地上的水流仍在流动。

相见：坤卦（地），坎卦（水），代表地上的水流仍在流动。

至诚：坤卦（地），坎卦（水），代表地上的水流仍在流动。

谦易：初盘：坤卦（地），坎卦（水），代表地上的水流仍在流动。

相见：坤卦（地），坎卦（水），代表地上的水流仍在流动。

至诚：坤卦（地），坎卦（水），代表地上的水流仍在流动。

地水师卦卦象变化代表了一个适应环境变化、顺应地性的态度，提示人们利用水流的力量，将水流引入地面，从而发挥其作用。这提醒我们在面对复杂的社会环境时，要善于观察和适应，利用现有的资源和条件，积极争取主动，争取美好的人生。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由三卦组成的卦象，上卦为乾（天），下卦为坎（水），中卦为坤（地），这表示了天、地、水三者的力量。在卜卦时，这个卦象表示了天神、地神和人类的联合，预示着卜者将会得到众神的帮助。

在卜卦时，解卦师需要结合卦象和卦辞来分析这个卦象所代表的含义。根据《易经》，天象征刚健，地象征柔顺，水象征险恶。因此，解卦师需要考虑到这一象征意义，分析卦象的组合是否和谐，从而得出正确的解卦。

对于地水师卦来说，重点是取刚健之刚，险恶之险，代表刚健的刚，用于险恶的险，这样就能

In [13]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
卦象为天priority（优先）+ 坎（水），代表天时地利人和都处于有利于我的境地，宜于采取行动。

HSH：天时地利人和都处于有利于我的境地，宜于采取行动。

ES：卦象表示我必须坚持正道，防止陷入别人设置的陷阱和诡计。

AI：宜于等待时机，勿冒进，勿急功近利。

ES：卦象告诉我，等待时机，勿冒进，勿急功近利。

LX：卦象表示我必居安思危，谨慎从事，创造美好未来。

D.：. .

Q：. .

D：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. .

.：. 

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [15]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [16]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，包含着天体运行的规律。它由两个阳爻（代表天）叠加而成，其中一个阳爻在位尊位置，称为天行健。另一阳爻在位谦卑位置，称为天唯敬。乾卦象征着天体运行的规律，强调刚强、积极、自信和果敢。

乾卦的卦辞是：“元、亨、利、贞”，这四个字意味着：元，始，代表万事万物的起始；亨，通，代表事物发展的顺利；利，代表事业成功；贞，正，代表事物运行的规律。综合起来，乾卦强调的是刚强、积极、自信和果敢，具有强烈的个人主义色彩。

在乾卦中，消息（指运动），阳动阴静，这是天行健的体现。在乾卦中，阳爻代表刚强和果敢，阴爻代表柔顺和谦卑。在《易传》中，阳爻表示天，阴爻表示地，天地相配合，代表万事万物都原始的形势。

在乾卦的险象中，有"天行健"的观念，表示天总是刚强地运行， without any stop，没有停滞，地行健，君子自强不息。

总结起来，乾卦是周易中的一卦，代表天，包含着天体运行的规律。它强调刚强、积极、自信和果敢，具有强烈的个人主义色彩。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，具有刚健的象征。它由两个乾卦叠加而成，其中上卦为乾，下卦为乾。在卜卦时，上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。在卜卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。在卜卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。在卜卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。在卜卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。

在卜卦时，乾卦表示刚健不衰的天运，预示着天象刚健，刚劲不衰。在人际关系中，代表着的天时刚健，对于自己的事业，预示着刚劲不衰。在决策中，刚强不衰的运筹，需要刚健的决策。

在解卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。在卜卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。在卜卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。在卜卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。在卜卦时，取上卦为乾，下卦为乾，表示天运刚健，刚劲不衰。

在解卦时，刚健不衰的天时，对于自己的事业，预示着刚劲不衰。在决策中，刚强不衰的运筹，需要刚健的决策。

在解卦时，刚强不衰的天时，对于自己的事业，预示着刚劲不衰。在决策中，刚强不衰的运筹，需

In [17]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由坤卦（地）和坎卦（水）组合而成，代表湖泽之象。在卜问中，预示着利卦，但也有忧吝之象。君子观此卦象，商议湖泽之利，从而得到好处，但必须谨慎从事。这一卦的核心哲学是：得到好处需要努力争取，但需注意不要沉迷于水中，失去控制。

师卦的哲学意义可以理解为：追求财富和权力，但需谨慎从事。在事业和生活中，可能需要领导众人，引导他们前进。在决策时，需要周详考虑，不能轻率行事。

师卦的应用范围广泛，可以预测顺利顺利捏合捏合情况，但需要谨防困难。在事业和生活中，指示正确的方向，但需要注意稳健和谨慎。在决策时，需要周详考虑，不能轻率行事。

师卦的哲学原则是：得到好处需要努力争取，但需注意不要沉迷于水中，失去控制。在事业和生活中，可能需要领导众人，引导他们前进。在决策时，需要周详考虑，不能轻率行事。

在解卦时，需要结合卦象和问卦，综合分析，得出正确的解释。在决策时，需要根据实际情况，权衡利弊，做出最佳选择。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由三卦组成的卦象，上卦为乾（天），下卦为坎（水），中卦为坤（地），这表示了天、地、水三者的力量。在卜卦时，这个卦象表示了天神、地神和人类的联合，预示着卜者将会得到众神的帮助。

在卜卦时，解卦师需要结合卦象和卦辞来分析这个卦象所代表的含义。根据《易经》，天象征刚健，地象征柔顺，水象征险恶。因此，解卦师需要考虑到这一象征意义，分析卦象的组合是否和谐，从而得出正确的解卦。

对于地水师卦来说，重点是取刚健之刚，险恶之险，代表刚健的刚，用于险恶的险，这样就能得到刚柔相济，刚强相互补充的结果。这样的卦象能够带来刚柔之间的力量，能够带来变化和成功。

地水师卦的解卦为刚健，代表天神的力量，带来刚强和自信。地水师卦的解卦为险恶，代表地神的力量，带来谨慎和谨慎。因此，解卦师需要综合分析天神和地神的自己的力量，从而得出正确的解卦。

在解卦时，需要结合卦象和卦辞来分析这个卦象所代表的含义。根据《易经》，天象征刚健，地象征柔顺，水象征险恶。因此，解卦师需要考虑到这一象征意义，分析卦象的组合是否和谐，从而得出正确的解卦。

对于地水师卦来说，重点是取刚健之刚，险恶之险，代表刚戴之戴，用于险恶的险，这样就能得到刚柔相济，刚强相互补充的结果。这样的卦

In [18]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，卦象为上坎下离，代表利见 Can（坎），止于正南（离），象征事情顺利，指数为1（初），为事物发展的开始。

讼卦的核心哲学是“利见Can，止于正南”，强调事物发展顺利， index为1（初），表示开始时顺利，但最终会遭遇挫折（Can），事情不会一直顺利下去。

讼卦的专业解释为：“讼卦，卦象为上坎下离，指数为1（初），为事物发展的开始。讼卦核心哲学是“利见Can，止于正南”，强调事物发展顺利， index为1（初），表示开始时顺利，但最终会遭遇挫折（Can），事情不会一直顺利下去。讼卦最终是有利起来的卦象，但必须要注意选择正确的方向和策略。”


ChatGLM3-6B微调后：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易中的一卦，代表诉讼、打斗的卦象，是由上卦坎（水）和下卦乾（天）组成的，象征着天坎之象。根据《象辞》，讼卦意味着天网恢恢，疏而不漏，小花小草都能在国度中成长，小鸟能在树梢上飞腾，小民能安国邦。根据哲学家的解卦，讼卦被认为是不利可利的卦象，提醒人们要避免争端，谨慎待人。在卜筮时，讼卦提醒人们要 statements


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [6]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240118_164514"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [8]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [9]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，由两个阳爻夹一个阴爻构成，象征着天。它代表着刚健、健行、刚健不屈的意义。乾卦象征刚健、健行、坚毅的品质，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在年第uck博士的风水flexibility中，乾卦也象征着天，代表着刚健、健行、坚毅的性格，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在风水学中，乾卦代表天，象征着坚毅、刚健、不屈不挠的品质，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在的风水flexibility中，乾卦也象征着天，代表着坚毅、健行、坚毅的性格，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在风水学中，乾卦代表天，象征着坚毅、健行、不屈不挠的品质，预示著前进。在事业、经商、求名等方面都可以看到乾卦的影子。在风


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [10]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
{'name': '其主要含义是占卜土地和水源的适宜与否，以及它们对人类生活的影响。在占卜中，地水师卦代表一种吉祥的卦象，预示着吉利和繁荣。', 'content': '\n在事业方面，地水师卦预示着事业发展顺利，提示要谨慎行事，行事光明磊落，的字面意思与水有关，表明大吉。\n\n在经商方面，虽然是吉利卦象，但需要谨慎行事，特别是在 business 交往中要谨慎行事，不能轻信他人，防止煮熟的鱼喂鸟，预防意外事件，不能贪图眼前利益，必须谨慎行事。\n\n在求名方面，提示要诚实、勤奋学习，努力提高自己的能力，与世无争，自然成功。\n\n在决策方面，提示要谨慎行事，不能草率行事，必须谨慎研究，根据实际情况制定计划，不能贪图眼前利益而忽视长远利益。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 卦名：地水师卦是一个由三个卦相组合而成，代表着自然界的力量和人类社会的活动。第一个卦是震卦，代表着震动和雷，第二个卦是坤卦，代表着顺从和大地，第三个卦是兑卦，代表着欢愉和快乐。地水师卦的核心哲学是：自然界的力量和人类社会的活动相互影响，人们应该顺应自然，发挥自己的才能，以达到和谐共处的境界。

卦象：震卦为雷，卦象是地震，代表着震动和雷。兑卦为泽，卦象是湖泊，代表着欢愉和快乐。坤卦为地，卦象是坤卦，代表着顺从和大地。

卦象解释：地水师卦的核心哲学是：自然界的力量和人类社会的活动相互影响，人们应该顺应自然，发挥自己的才能，以达到和谐共处的境界。

君子观象：君子应当学习震卦的震动和兑卦的快乐，从而返回震卦的震动。


In [11]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。在周易中，坎代表着险恶，而乾则象征着高洁。这一卦象预示着天地之间的险恶已经来到人们中间，但天依然高悬，没有改变。对于创作者来说，这一卦象提醒了的风险和挑战，然而也给了创作者展望未来的信心。

讼卦的核心哲学是：险象环生，但仍有希望。和高洁的乾卦相比，坎卦代表着险恶，但高洁的乾卦仍然在上面，意味着高洁之志未尝失。因此，尽管讼卦充满风险和挑战，但仍有希望。

对于创作者来说，尽管面临各种风险和挑战，但仍有希望。但需高洁的品质，坚定信念，克服困难，但也需谨慎行事，避免险恶之险。
